In [2]:
!pip install s3fs
!pip install numpy
!pip install pandas

In [6]:
import os
import math
from typing import List

import numpy as np
import pandas as pd

print("pandas version:", pd.__version__)

# s3 기본 설정
# BUCKET = "sagemaker-us-west-2-327784329358"
# S3_BASE = f"s3://{BUCKET}"

# 입력 데이터: S3에서 바로 읽기
# PRICE_OHLCV_PATH = f"{S3_BASE}/prepared_data/price_ohlcv.csv"
# VQ_CODE_CSV_PATH = f"{S3_BASE}/vq_vae_outputs/vq_codes_spy.csv"
# NEWS_CSV_PATH = f"{S3_BASE}/prepared_data/sp500_headlines_2008_2024.csv"

# 출력 JSONL: 로컬에 만들고 S3로 업로드
# OUTPUT_JSONL_LOCAL_PATH = "/tmp/market_commentary_train.jsonl"
# OUTPUT_JSONL_S3_KEY = "prepared_data/market_commentary_train.jsonl"

# Data Path
OUTPUT_JSONL_LOCAL_PATH = "prepared_data/train.jsonl"
SILVER_LABEL_PATH = "silver_label_market_trend.csv"
PRICE_OHLCV_PATH = "spy_2023_2024.csv"

pandas version: 2.3.3


In [22]:
df_silver_label = pd.read_csv(SILVER_LABEL_PATH)
df_price = pd.read_csv(PRICE_OHLCV_PATH)

In [23]:
# required_cols = {"date", "close"}
# if not required_cols.issubset(df_price.columns):
    # raise ValueError("prices_ohlcv.csv에 date, close 컬럼이 필요합니다.")

df_silver_label['date'] = pd.to_datetime(df_silver_label['date']).dt.date

df_price['date'] = pd.to_datetime(df_price['date'])
df_price['day'] = df_price['date'].dt.date
df_price = df_price.groupby('day').tail(1)

def make_summary(row):
    return (
        f"open: {row['1. open']}, "
        f"high: {row['2. high']}, "
        f"low: {row['3. low']}, "
        f"close: {row['4. close']}, "
        f"volume: {row['5. volume']}"
    )

df_price["ohlcv_summary"] = df_price.apply(make_summary, axis=1)
df_price = df_price[['day', 'ohlcv_summary']].copy()
df_price = df_price.rename(columns={'day': 'date'})
df_price['date'] = pd.to_datetime(df_price['date']).dt.date

# df_price["date"] = pd.to_datetime(df_price["date"])
# df_price = df_price.sort_values("date").reset_index(drop=True)

# 여기서 수익률, 변동성 등 파생 피처 생성
# df_price["ret_1d"] = df_price["close"].pct_change()
# df_price["log_ret"] = np.log(df_price["close"]).diff()
# df_price["close_ma_5"] = df_price["close"].rolling(window=5).mean()
# df_price["close_ma_20"] = df_price["close"].rolling(window=20).mean()
# df_price["vol_5"] = df_price["log_ret"].rolling(window=5).std()
# df_price["vol_20"] = df_price["log_ret"].rolling(window=20).std()

# NaN 제거
# df_price = df_price.dropna().reset_index(drop=True)
# print("[PRICE] head:\n", df_price.head())


# VQ-VAE 코드와 price join
# df_vq = pd.read_csv(VQ_CODE_CSV_PATH)
# df_vq["date"] = pd.to_datetime(df_vq["date"])

# df_merged = pd.merge(df_vq, df_price, on="date", how="inner")
# df_merged = df_merged.sort_values("date").reset_index(drop=True)

# News Headline join
# df_news = pd.read_csv("NEWS_CSV_PATH")
# df_news["Date"] = pd.to_datetime(df_news["Date"])

""" news_grouped = (
    df_news.groupby("Date")["Title"]
    .apply(list)
    .reset_index()
    .rename(columns={"Date": "date", "Title": "titles"})
) """

# merge all
df_all = pd.merge(
    df_silver_label,
    df_price,                # VQ 코드 + 가격 요약
    on="date",
    how="left",
)

In [31]:
def code_to_str(row):
    z_values = [row[f"z_{i}"] for i in range(8)]
    z_str = ", ".join([f"{v:.6f}" for v in z_values])
    return f"{z_str}"

df_all["code_str"] = df_all.apply(code_to_str, axis=1)

In [32]:
df_all.head()

,date,code,z_0,z_1,z_2,z_3,z_4,z_5,z_6,z_7,text,ohlcv_summary,prompt,completion,code_str
0,2023-01-03,25,0.021594,0.189350,0.005870,0.094773,-9.261945e-28,0.034962,0.401183,0.569945,"핵심 주제: 기업 경영, 시장 동향 및 신기술\n다양한 기업들이 경영진 개편, M&...","open: 366.6838, high: 367.5803, low: 366.6066,...","### 코드\n25\n\n### OHLCV 시계열\nopen: 366.6838, h...","핵심 주제: 기업 경영, 시장 동향 및 신기술\n다양한 기업들이 경영진 개편, M&...","0.021594, 0.189350, 0.005870, 0.094773, -0.000..."
1,2023-01-04,12,0.102584,0.253331,0.088687,0.262031,1.646256e-24,0.050349,0.120076,0.256553,핵심 주제: 2023년 초 글로벌 비즈니스 및 기술 동향\n\n2023년 초 다양한...,"open: 368.9878, high: 370.1735, low: 367.4996,...","### 코드\n12\n\n### OHLCV 시계열\nopen: 368.9878, h...",핵심 주제: 2023년 초 글로벌 비즈니스 및 기술 동향\n\n2023년 초 다양한...,"0.102584, 0.253331, 0.088687, 0.262031, 0.0000..."
2,2023-01-05,9,0.034468,0.032504,0.005644,0.347086,-1.079616e-28,0.339522,0.439623,0.407905,핵심 주제: 2023년 초 다양한 산업의 혁신 및 시장 동향: CES 2023을 통...,"open: 365.9704, high: 366.2837, low: 365.2811,...","### 코드\n9\n\n### OHLCV 시계열\nopen: 365.9704, hi...",핵심 주제: 2023년 초 다양한 산업의 혁신 및 시장 동향: CES 2023을 통...,"0.034468, 0.032504, 0.005644, 0.347086, -0.000..."
3,2023-01-06,7,0.190114,0.087750,0.160866,0.387595,-1.205483e-27,0.435344,0.160252,0.076243,"핵심 주제: 광범위한 산업의 시장 성장 전망, 기업의 재무 성과, 투자 및 자금 조...","open: 374.4007, high: 375.0707, low: 373.7789,...","### 코드\n7\n\n### OHLCV 시계열\nopen: 374.4007, hi...","핵심 주제: 광범위한 산업의 시장 성장 전망, 기업의 재무 성과, 투자 및 자금 조...","0.190114, 0.087750, 0.160866, 0.387595, -0.000..."
4,2023-01-09,16,0.111109,0.432911,0.087223,0.046769,5.278745e-22,0.038995,0.109769,0.296704,핵심 주제: 글로벌 기업 경영 활동 및 시장 전망. 다양한 산업 분야에서 기업들의 ...,"open: 374.3863, high: 374.5694, low: 373.7211,...","### 코드\n16\n\n### OHLCV 시계열\nopen: 374.3863, h...",핵심 주제: 글로벌 기업 경영 활동 및 시장 전망. 다양한 산업 분야에서 기업들의 ...,"0.111109, 0.432911, 0.087223, 0.046769, 0.0000..."


In [39]:
def build_prompt(code_value: int, code_vector: str, ohlcv_summary: str):
    prompt = (
        "### 코드북\n" + str(code_value) +"\n\n"
        "### 코드북 벡터\n" + str(code_vector) + "\n\n"
        "### OHLCV 시계열\n" + ohlcv_summary + "\n\n"

        "위의 Code 벡터 정보와 OHLCV 시계열을 바탕으로, 해당 구간의 시장 움직임을 설명할 수 있는 한국어 뉴스 요약을 2~3 문장으로 작성하라."
        "데이터에 기반한 보수적 설명만 사용하고, 입력에 없는 사건을 새로 만들어내지 마라."
    )
    return prompt
    
# 프롬프트 컬럼 생성 (레이블은 placeholder)
df_all["prompt"] = df_all.apply(
    lambda row: build_prompt(
        code_value=row["code"],
        code_vector=row["code_str"],
        ohlcv_summary=row["ohlcv_summary"],
    ),
    axis=1,
)

# 실제 프로젝트에서는 아래 completion을 사람/teacher LLM으로 채워넣어야 함
# silver label을 llm api로 사용하거나 사람 손으로 gole label을 붙여 넣어야 함
df_all["completion"] = df_all["text"]

print("[ALL with prompt/completion] head:\n",
      df_all[["date", "code", "prompt", "completion"]].head())


[ALL with prompt/completion] head:
          date  code                                             prompt  \
0  2023-01-03    25  ### 코드북\n25\n\n### 코드북 벡터\n0.021594, 0.189350,...   
1  2023-01-04    12  ### 코드북\n12\n\n### 코드북 벡터\n0.102584, 0.253331,...   
2  2023-01-05     9  ### 코드북\n9\n\n### 코드북 벡터\n0.034468, 0.032504, ...   
3  2023-01-06     7  ### 코드북\n7\n\n### 코드북 벡터\n0.190114, 0.087750, ...   
4  2023-01-09    16  ### 코드북\n16\n\n### 코드북 벡터\n0.111109, 0.432911,...   

                                          completion  
0  핵심 주제: 기업 경영, 시장 동향 및 신기술\n다양한 기업들이 경영진 개편, M&...  
1  핵심 주제: 2023년 초 글로벌 비즈니스 및 기술 동향\n\n2023년 초 다양한...  
2  핵심 주제: 2023년 초 다양한 산업의 혁신 및 시장 동향: CES 2023을 통...  
3  핵심 주제: 광범위한 산업의 시장 성장 전망, 기업의 재무 성과, 투자 및 자금 조...  
4  핵심 주제: 글로벌 기업 경영 활동 및 시장 전망. 다양한 산업 분야에서 기업들의 ...  


In [40]:
# import boto3
import json

def export_jsonl(df: pd.DataFrame, out_path: str) -> None:
    os.makedirs(os.path.dirname(out_path) or ".", exist_ok=True)
    with open(out_path, "w", encoding="utf-8") as f:
        for row in df.itertuples():
            rec = {
                "prompt": getattr(row, "prompt"),
                "completion": getattr(row, "completion"),
                "date": str(getattr(row, "date")),
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    print(f"Saved JSONL to {out_path}")

export_jsonl(df_all, OUTPUT_JSONL_LOCAL_PATH)

# upload to s3 bucket
# s3 = boto3.client("s3")
# s3.upload_file(OUTPUT_JSONL_LOCAL_PATH, BUCKET, OUTPUT_JSONL_S3_KEY)

# print(f"Uploaded to s3://{BUCKET}/{OUTPUT_JSONL_S3_KEY}")

Saved JSONL to prepared_data/train.jsonl
